In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
PATH = '/Users/ndminh/Minh/USA/University/University\ of\ Houston/Study/Spring\ 2023/COSC\ 3337/Kaggle/01-27-23/cosc3337-sp23-comp-1-roam-in-dubai-home/'
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Read the data
train = pd.read_csv("/kaggle/input/cosc3337-sp23-comp-1-roam-in-dubai-home/train.csv")
test = pd.read_csv("/kaggle/input/cosc3337-sp23-comp-1-roam-in-dubai-home/test.csv")

**Why we need to take a look at both the training and the test data ?**

Well, as long as we are not "seeing" the prediction on the test data, we are good :)

But it will make your life a bit easier in dealing with the "pipelined" tasks. 

In [ ]:
train.head()

In [ ]:
#check the shape
train.shape

In [ ]:
# what are the datatypes
train.dtypes

In [ ]:
# Take a quick idea on all columns
train.describe(include="all")

In [ ]:
## The target column is the "Price". So, let's get rid of that 
train_without_price = train.drop(["price"], axis=1)
train_without_price.shape

In [ ]:
test.shape

In [ ]:
# Now I want to look at the target variable. Since, I want to use linear regression,
# if the data doesn't look normalized, it will be a BAD NEWS for me

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(train.price.values)

In [ ]:
# My hunch was true. Now, to make them look normalized, I will do a log transformation 
log_price = np.log(train.price.values)
sns.distplot(log_price)

In [ ]:
    combined = pd.concat([train_without_price, test], axis=0)


In [ ]:
# You cannot work with object data (string) directly. You need to convert them to "dummy variables" or one-hot-encoded version
combined_obj = combined[["neighborhood", "quality"]]
combined_obj = pd.get_dummies(combined_obj)
combined_obj.head()

In [ ]:
# Now, drop the unwanted columns from the main train set
combined_nonObj = combined.drop(["Unnamed: 0", "id", "neighborhood", "quality"], axis=1)
combined_nonObj.head()

In [ ]:
# Finally concatenate the transformed data
combined_conc = pd.concat([combined_obj, combined_nonObj], axis=1)
combined_conc.head()

In [ ]:
combined_conc.shape

In [ ]:
# Now, let's differentiate train and the test
train_final = combined_conc[:train.shape[0]]
test_final = combined_conc[train.shape[0]:]

In [ ]:
# Now, let's focus on the train set. 
y = log_price
X = train_final.values
print("feature shape is", X.shape)
print("target shape is", y.shape)

**Now, let's build our regression model**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt

In [ ]:
# create a linear regression model
reg = LinearRegression().fit(X, y)

In [ ]:
pred = reg.predict(X)

In [ ]:
math.sqrt(mean_squared_error(y, pred)) 


In [ ]:
#But that's not the original price. The RMSE on actual price would be

In [ ]:
math.sqrt(mean_squared_error(np.exp(y), np.exp(pred))) 


The way I have created it, just for a demo purpose. You can use any algorithm and try to minimize the error

In [ ]:
plt.scatter(pred, y)
plt.xlabel("Predicted Price")
plt.ylabel("Actual Price")
plt.title("On Log price")
plt.show()

In [ ]:
plt.scatter(np.exp(pred), np.exp(y))
plt.xlabel("Predicted Price")
plt.ylabel("Actual Price")
plt.title("On original price")
plt.show()

In [ ]:
# Now we will do the actual prediction on test set
test_X = test_final.values
pred = reg.predict(test_X)
pred.shape

In [ ]:
pred_org = np.exp(pred)

In [ ]:
### Please follow the guidelines here. The "Unnamed: 0" is the "sid" column for your result
df = pd.DataFrame()
df["sid"] = test["Unnamed: 0"]
df["price"] = pred_org
df.to_csv("sub_1.csv", index=False)

In [ ]:
df